<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1Predict_by_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret==2.3.1

In [2]:
from pycaret.utils import enable_colab

enable_colab()

Colab mode enabled.


In [17]:
import numpy as np
import pandas as pd
import time

csv = '/content/seasonData_2021.csv'
data = pd.read_csv(csv)
data = data.drop(data.columns[[0]], axis=1)
data.shape

(380, 28)

# データの加工

In [18]:
data['CBP_x'] = data['CBP_x'].str.replace('%', '').astype(np.float64) /100
data['shot acc_x'] = data['shot acc_x'].str.replace('%', '').astype(np.float64) / 100
data['possession_x'] = data['possession_x'].str.replace('%', '').astype(np.float64) / 100
data['CBP_y'] = data['CBP_y'].str.replace('%', '').astype(np.float64) / 100
data['shot acc_y'] = data['shot acc_y'].str.replace('%', '').astype(np.float64) / 100
data['possession_y'] = data['possession_y'].str.replace('%', '').astype(np.float64) / 100

# pandas profiling

In [ ]:
!pip install --upgrade pandas-profiling

In [ ]:
import pandas_profiling as pdp
pdp.ProfileReport(data).to_file(output_file = 'profileReport.html')

# SetUP

In [5]:
!pip install pyyaml==5.4.1

In [19]:
from pycaret.regression import setup
data = data.drop('節', axis=1)
data = data.drop('開催日', axis=1)
def x(a,b):
    return a - b
data['homeScore']  = data.apply(lambda f: x(f['homeScore'],f['awayScore']), axis=1)
data = data.drop('awayScore', axis=1)
data = data.rename(columns={'homeScore': 'Score'})
data = data.drop('H or A_x',axis = 1)
data = data.drop('H or A_y',axis = 1)
data

,Team,awayTeam,Score,AGI_x,KAGI_x,CBP_x,shot_x,shot acc_x,possession_x,ACBP_x,PassCBP_x,InterceptP_x,DP_x,AGI_y,KAGI_y,CBP_y,shot_y,shot acc_y,possession_y,ACBP_y,PassCBP_y,InterceptP_y,DP_y
0,広島,仙台,0,70,59,0.176,19,0.053,0.576,23.40,19.60,83.02,10.45,41,30,0.061,7,0.143,0.424,9.07,7.21,82.69,17.97
1,神戸,Ｇ大阪,1,12,51,0.063,8,0.125,0.582,11.54,10.22,118.05,13.65,49,88,0.061,8,0.000,0.418,12.28,10.08,101.88,12.44
2,大分,徳島,0,42,49,0.094,10,0.100,0.374,8.78,6.79,93.51,18.91,51,58,0.126,13,0.077,0.626,17.04,13.42,99.86,10.39
3,福岡,名古屋,-1,50,41,0.033,4,0.000,0.505,13.11,10.11,113.83,11.57,59,50,0.081,10,0.200,0.495,13.29,10.12,105.37,14.75
4,川崎Ｆ,横浜FM,2,57,44,0.137,18,0.111,0.469,15.98,14.30,139.34,13.68,56,43,0.082,11,0.000,0.531,17.02,15.60,108.41,10.76
5,Ｃ大阪,柏,2,57,56,0.169,20,0.100,0.552,18.87,12.12,94.30,11.08,44,43,0.097,10,0.000,0.448,11.43,9.70,74.28,16.31
6,鹿島,清水,-2,60,70,0.173,23,0.043,0.560,20.28,16.30,129.83,10.57,30,40,0.066,9,0.222,0.440,8.70,6.70,115.33,22.38
7,湘南,鳥栖,-1,70,53,0.070,9,0.000,0.460,13.09,10.28,122.23,10.96,47,30,0.078,10,0.100,0.540,12.61,11.36,116.86,17.86
8,札幌,横浜FC,4,61,66,0.209,23,0.217,0.543,24.11,18.98,131.07,11.56,34,39,0.033,4,0.250,0.457,8.16,6.68,84.89,14.57
9,浦和,FC東京,0,57,47,0.086,11,0.091,0.587,14.87,12.92,86.11,12.97,53,43,0.042,5,0.200,0.413,8.16,8.05,121.59,17.13


In [20]:
setup = setup(data, target="Score",
        session_id = 123)

,Description,Value
0,session_id,123
1,Target,Score
2,Original Data,"(380, 23)"
3,Missing Values,False
4,Numeric Features,20
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(265, 57)"


In [21]:
setup

(5,
 False,
 KFold(n_splits=10, random_state=None, shuffle=False),
 None,
 False,
 {'mae': <pycaret.containers.metrics.regression.MAEMetricContainer at 0x7f11b9ab6f10>,
  'mape': <pycaret.containers.metrics.regression.MAPEMetricContainer at 0x7f11b9ab9110>,
  'mse': <pycaret.containers.metrics.regression.MSEMetricContainer at 0x7f11b9ab6f50>,
  'r2': <pycaret.containers.metrics.regression.R2MetricContainer at 0x7f11b9ab6fd0>,
  'rmse': <pycaret.containers.metrics.regression.RMSEMetricContainer at 0x7f11b9ab6f90>,
  'rmsle': <pycaret.containers.metrics.regression.RMSLEMetricContainer at 0x7f11b9ab9050>},
 Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='Score',
                                      

In [22]:
from pycaret.regression import compare_models

best_model = compare_models()  # 全てのモデルを訓練し、評価する

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,4.286000e-01,4.196000e-01,6.283000e-01,8.650000e-01,0.0,2.767000e-01,0.122
rf,Random Forest Regressor,4.894000e-01,5.894000e-01,7.409000e-01,8.141000e-01,0.0,3.062000e-01,0.521
et,Extra Trees Regressor,4.708000e-01,5.652000e-01,7.353000e-01,8.133000e-01,0.0,2.959000e-01,0.470
omp,Orthogonal Matching Pursuit,5.341000e-01,6.321000e-01,7.704000e-01,7.885000e-01,0.0,3.541000e-01,0.015
lr,Linear Regression,6.225000e-01,7.031000e-01,8.219000e-01,7.616000e-01,0.0,4.112000e-01,0.353
br,Bayesian Ridge,6.365000e-01,7.094000e-01,8.295000e-01,7.585000e-01,0.0,4.143000e-01,0.019
ada,AdaBoost Regressor,6.659000e-01,8.281000e-01,8.930000e-01,7.354000e-01,0.0,4.310000e-01,0.121
ridge,Ridge Regression,7.028000e-01,8.277000e-01,8.993000e-01,7.159000e-01,0.0,4.474000e-01,0.014
dt,Decision Tree Regressor,6.905000e-01,1.225500e+00,1.098400e+00,5.791000e-01,0.0,4.577000e-01,0.337
huber,Huber Regressor,1.034100e+00,1.866900e+00,1.348900e+00,3.690000e-01,0.0,6.421000e-01,0.059


In [23]:
print(best_model)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=123, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


# Create Model

In [24]:
from pycaret.regression import models

models()  # 機械学習モデルの一覧

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [25]:
from pycaret.regression import create_model

gbr = create_model("gbr")  # GradientBoostingRegressorのモデルを作成

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.2929,0.2008,0.4481,0.9014,-0.0,0.2379
1,0.3655,0.2718,0.5213,0.8741,-0.0,0.2704
2,0.4809,0.4955,0.7039,0.8454,-0.0,0.2683
3,0.3459,0.1859,0.4312,0.9388,-0.0,0.2323
4,0.6178,0.9171,0.9576,0.7878,-0.0,0.2770
5,0.3941,0.3250,0.5701,0.8868,-0.0,0.2963
6,0.3438,0.2737,0.5231,0.8697,-0.0,0.2586
7,0.4985,0.5911,0.7688,0.7605,-0.0,0.3326
8,0.5007,0.5702,0.7551,0.8581,-0.0,0.3512
9,0.4456,0.3647,0.6039,0.9276,-0.0,0.2420


In [26]:
print(gbr)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=123, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


# Tune Model

In [27]:
from pycaret.regression import tune_model

tuned_gbr = tune_model(gbr)  # ハイパーパラメータの調整

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.3262,0.2274,0.4769,0.8883,-0.0,0.2848
1,0.3692,0.2962,0.5442,0.8628,-0.0,0.3018
2,0.4171,0.3109,0.5576,0.9030,-0.0,0.2336
3,0.3564,0.1882,0.4338,0.9380,-0.0,0.2547
4,0.5347,0.7720,0.8786,0.8213,-0.0,0.2506
5,0.3627,0.2509,0.5009,0.9126,-0.0,0.3013
6,0.4535,0.5397,0.7346,0.7431,-0.0,0.2717
7,0.4634,0.4980,0.7057,0.7982,-0.0,0.2891
8,0.5400,0.7274,0.8529,0.8190,-0.0,0.3645
9,0.4788,0.4134,0.6429,0.9179,-0.0,0.2720


In [28]:
print(tuned_gbr)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.15, loss='ls', max_depth=7,
                          max_features=1.0, max_leaf_nodes=None,
                          min_impurity_decrease=0.02, min_impurity_split=None,
                          min_samples_leaf=5, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=230,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=123, subsample=0.85, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


# evalation

In [29]:
from pycaret.regression import evaluate_model

evaluate_model(tuned_gbr)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

# deploy

In [30]:
from pycaret.regression import finalize_model

final_gbr = finalize_model(tuned_gbr)
print(final_gbr)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.15, loss='ls', max_depth=7,
                          max_features=1.0, max_leaf_nodes=None,
                          min_impurity_decrease=0.02, min_impurity_split=None,
                          min_samples_leaf=5, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=230,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=123, subsample=0.85, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


In [31]:
from pycaret.regression import predict_model

predict_model(final_gbr);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0357,0.0025,0.0496,0.999,0.0264,0.0263


In [32]:
from pycaret.regression import save_model

save_model(final_gbr, "gbr_model")

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='Score',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy='...
                                            learning_rate=0.15, loss='ls',
                                            max_depth=7, max_features=1.0,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.02,
                          

In [33]:
from pycaret.regression import load_model

loaded_gbr = load_model("gbr_model")

Transformation Pipeline and Model Successfully Loaded


# get Pred Data

In [ ]:
#試合日は以下を例に入力
#2021-8-9-15:00
def scrapePredData(homeTeamID,awayTeamID,year,matchDate):
    data = pd.DataFrame()
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チーム毎の平均スタッツを取得
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0,2,4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)

    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'試合平均': 'home','Unnamed: 3': 'Data','試合平均.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    time.sleep(1)
    #チーム名と試合日
    homeTeam = get_keys_from_value(TeamDic,homeTeamID)
    awayTeam = get_keys_from_value(TeamDic,awayTeamID)
    df.insert(0, 'チーム名',[homeTeam,awayTeam])
    df.insert(0, '試合日', matchDate)
    data = pd.concat([data, df]) 
    data['総移動距離'] = data['総移動距離'].str.replace('km', '').astype(np.float64)
    data['相手との移動距離差'] = data['相手との移動距離差'].str.replace('km', '').astype(np.float64)
    data['チャンス構築率'] = data['チャンス構築率'].str.replace('%', '').astype(np.float64)
    data['成功率'] = data['成功率'].str.replace('%', '').astype(np.float64)
    data['被成功率'] = data['被成功率'].str.replace('%', '').astype(np.float64)
    data['被チャンス構築率'] = data['被チャンス構築率'].str.replace('%', '').astype(np.float64)

    return data

### Team Dic

In [ ]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

### ここのTeamDicと日付を変えれば予想できる

In [ ]:
pred_data = scrapePredData(TeamDic['鳥栖'],TeamDic['名古屋'],'2021','2021-8-21-19:00')
filename =  'predData' + '.csv'
pred_data.to_csv(filename, encoding = 'utf-8-sig') 
pred_data = pd.read_csv('/content/predData.csv')
pred_data

,Unnamed: 0,試合日,チーム名,攻撃,パス,クロス,ドリブル,シュート,ゴール,奪取,守備,セーブ,攻撃回数,シュート.1,チャンス構築率,ゴール.1,成功率,被攻撃回数,被シュート,被チャンス構築率,被ゴール,被成功率,ゴール期待値,ゴール.2,被ゴール期待値,被ゴール.1,総移動距離,相手との移動距離差,スプリント数,相手とのスプリント数差,ゴール.3,シュート.2,枠内シュート,パス.1,クロス.1,直接ＦＫ,間接ＦＫ,ＣＫ,スローイン,ドリブル.1,タックル,クリア,インターセプト,オフサイド,警告,退場,３０ｍライン進入,ペナルティエリア進入
0,home,2021-8-21-19:00,['鳥栖'],14.83,12.31,1.37,1.15,6.20,4.03,91.06,14.49,0.46,112.1,11.3,10.1,1.3,11.8,115.0,10.9,9.5,0.7,6.1,1.134,1.3,1.093,0.7,124.998,4.849,175.9,-10.3,1.3,11.3,3.6,535.9,11.0,8.8,1.5,3.8,21.4,9.0,17.6,21.7,1.8,2.3,1.1,0.1,37.9,10.0
1,away,2021-8-21-19:00,['名古屋'],13.44,11.00,1.22,1.21,5.66,2.52,96.64,12.63,0.25,122.4,9.9,8.1,0.9,9.2,118.8,9.3,7.9,0.7,7.6,0.911,0.9,0.807,0.7,116.247,0.559,175.6,2.0,0.9,9.9,3.5,491.5,11.8,14.2,1.3,4.2,25.1,12.9,18.0,20.8,2.2,1.4,0.9,0.0,34.8,9.6


# prediction

In [ ]:
pred_data = pred_data.drop('チーム名', axis=1)
#pred_data = pred_data.drop('Unnamed: 0',axis = 1)
prediction = predict_model(loaded_br, data=pred_data)

In [ ]:
print('Home',prediction['Label'][0],'vs','Away',prediction['Label'][1])

Home 1.2901192969875102 vs Away 1.0760561199948797
